# Zurich Instruments LabOne Python API Example

## Multiplexed read-out of 8 qubits

Perform the multiplexed read-out of 8 qubits in parallel.

Requirements:
* LabOne Version >= 22.08
* Instruments:
    1 x SHFQA or SHFQC Instrument
* Loopback configuration between input and output of channel 0

----

In [ ]:
import zhinst.core
import zhinst.utils.shfqa as shfqa_utils

### 1. Connect the device

Set up the connection with the device. Firstly create an API session with a Data Server. Then the Data Server connects to the device.

In [ ]:
device_id = "dev2345" # Device serial number available on its rear panel.
interface = "1GbE" # For Ethernet connection
#interface = "USB" # For USB connection

server_host = "localhost"
server_port = 8004
api_level = 6 # Maximum API level supported for all instruments.

# Create an API session to the Data Server.
daq = zhinst.core.ziDAQServer(server_host, server_port, api_level)

# Establish a connection between Data Server and Device.
daq.connectDevice(device_id, interface)

### 2. Configure the SHFQA channel

Define the parameters of the experiment.

In [ ]:
channel_index = 0
num_qubits = 8
num_readouts = 100

Configure the SHFQA channel.

In [ ]:
shfqa_utils.configure_channel(
    daq,
    device_id,
    channel_index,
    center_frequency=5e9,
    input_range=0,
    output_range=-5,
    mode="readout",
)

Turn on the input and output of the channel.

In [ ]:
daq.set(
    [
        (f"/{device_id}/qachannels/{channel_index}/input/on", 1),
        (f"/{device_id}/qachannels/{channel_index}/output/on", 1),
    ]
)

### 3. Generate and upload the waveforms and integration weights

Define the waveforms' parameters.

In [ ]:
import numpy as np

# Configuration parameter for the waveform
scaling = 0.9 / num_qubits
frequencies = np.linspace(32e6, 230e6, num_qubits)
pulse_duration = 500e-9
rise_fall_time = 10e-9
sampling_rate = shfqa_utils.SHFQA_SAMPLING_FREQUENCY
scaling = scaling

rise_fall_len = int(rise_fall_time * sampling_rate)
pulse_len = int(pulse_duration * sampling_rate)
std_dev = rise_fall_len // 10

Generate the waveforms.

In [ ]:
from scipy.signal import gaussian

# Create a flat top gaussian
gauss = gaussian(2 * rise_fall_len, std_dev)
flat_top_gaussian = np.ones(pulse_len)
flat_top_gaussian[0:rise_fall_len] = gauss[0:rise_fall_len]
flat_top_gaussian[-rise_fall_len:] = gauss[-rise_fall_len:]

flat_top_gaussian *= scaling

# Modulate complex sinusoids with flat top gaussian
readout_pulses = {}
time_vec = np.linspace(0, pulse_duration, pulse_len)

for i, f in enumerate(frequencies):
    readout_pulses[i] = flat_top_gaussian * np.exp(2j * np.pi * f * time_vec)

Upload the waveforms to memory.

In [ ]:
shfqa_utils.write_to_waveform_memory(
    daq, device_id, channel_index, waveforms=readout_pulses
)

Generate the integration weights.

In [ ]:
weights = {}
for waveform_slot, pulse in readout_pulses.items():
    weights[waveform_slot] = np.conj(pulse * np.exp(1j))

Configure the weighted integration.

In [ ]:
shfqa_utils.configure_weighted_integration(
    daq,
    device_id,
    channel_index,
    weights=weights,
    # compensation for the delay between generator output and input of the integration unit
    integration_delay=200e-9,
)
shfqa_utils.configure_result_logger_for_readout(
    daq,
    device_id,
    channel_index,
    result_source="result_of_integration",
    result_length=num_readouts,
)

Configure and load the sequencer.

In [ ]:
# Configure the sequence triggering
shfqa_utils.configure_sequencer_triggering(
    daq, device_id, channel_index, aux_trigger="software_trigger0"
)

# Define the sequencer program
seqc_program = f"""\
repeat({num_readouts})
{{
    waitDigTrigger(1);
    startQA(QA_GEN_ALL, QA_INT_ALL, true, 0, 0x0);
}}
"""

# Load the sequencer program
shfqa_utils.load_sequencer_program(
    daq, device_id, channel_index, sequencer_program=seqc_program
)


### 4. Run the experiment

Run the experiment.

> Note:
> 
> Software triggering (`start_continuous_sw_trigger()`) is used for illustration purposes only. Use a real
> trigger source for actual experiments.

In [ ]:
shfqa_utils.enable_result_logger(daq, device_id, channel_index, mode="readout")

shfqa_utils.enable_sequencer(daq, device_id, channel_index, single=1)

shfqa_utils.start_continuous_sw_trigger(
    daq, device_id, num_triggers=num_readouts, wait_time=2e-3
)


Get the data.

In [ ]:
readout_results = shfqa_utils.get_result_logger_data(
    daq, device_id, channel_index, mode="readout"
)

Plot the result.

In [ ]:
import matplotlib.pyplot as plt

max_value = 0

plt.rcParams["figure.figsize"] = [10, 10]

# Plot the results on real-imag plane
for complex_number in readout_results[:num_qubits]:
    real = np.real(complex_number)
    imag = np.imag(complex_number)

    plt.plot(real, imag, "x")

    max_value = max(max_value, max(abs(real)))
    max_value = max(max_value, max(abs(imag)))

# Zoom so that the origin is in the middle
max_value *= 1.05
plt.xlim([-max_value, max_value])
plt.ylim([-max_value, max_value])

plt.legend(range(num_qubits))
plt.title("qubit readout results")
plt.xlabel("real part")
plt.ylabel("imaginary part")
plt.grid()
plt.show()